In [102]:
import pandas as pd
import numpy as np
import sys
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin


In [103]:
class GuvenliEtiketleyici(BaseEstimator, TransformerMixin):
    """
    Scikit-Learn Pipeline ile uyumlu, güvenli bir LabelEncoder.
    Her sütun için ayrı bir LabelEncoder uygular ve eğitim setinde
    görülmeyen yeni kategorileri ('unknown') -1 olarak kodlar.
    """
    def __init__(self):
        self.kodlayicilar = {}
        self.sutunlar = None

    def fit(self, X, y=None):
        # Veri tipine göre sütun isimlerini belirle
        if isinstance(X, pd.DataFrame):
            self.sutunlar = X.columns
            X_dizi = X.values
        else:
            X_dizi = X
            self.sutunlar = [f"sutun_{i}" for i in range(X.shape[1])]
        
        # Her sütun için ayrı bir LabelEncoder eğit
        for i, sutun_adi in enumerate(self.sutunlar):
            kodlayici = LabelEncoder()
            # NaN değerlerini geçici bir kategoriyle değiştirerek uyumlu hale getir
            gecerli_veri = np.where(pd.isna(X_dizi[:, i]), 'EKSİK_DEGER', X_dizi[:, i].astype(str))
            kodlayici.fit(gecerli_veri)
            self.kodlayicilar[sutun_adi] = kodlayici
        return self

    def transform(self, X):
        X_dizi = X.values if isinstance(X, pd.DataFrame) else X
        sonuc = np.zeros_like(X_dizi, dtype=float)

        # Her sütunu eğitilmiş kodlayıcı ile dönüştür
        for i, sutun_adi in enumerate(self.sutunlar):
            kodlayici = self.kodlayicilar[sutun_adi]
            gecerli_veri = np.where(pd.isna(X_dizi[:, i]), 'EKSİK_DEGER', X_dizi[:, i].astype(str))
            
            # Bilinmeyen kategorileri -1, bilinenleri ise kendi kodlarıyla değiştir
            donusturulmus_sutun = [
                kodlayici.transform([deger])[0] if deger in kodlayici.classes_ else -1
                for deger in gecerli_veri
            ]
            sonuc[:, i] = donusturulmus_sutun
        return sonuc



In [105]:
class VeriOnIsleyici:
    """
    Veri ön işleme sürecini yöneten ana sınıf.
    Veriyi yükler, temizler ve bir Scikit-Learn Pipeline'ı aracılığıyla işler.
    """
    def __init__(self, dosya_yolu):
        self.dosya_yolu = dosya_yolu
        self.orijinal_veri = None
        self.hazirlanmis_veri = None
        self.tam_pipeline = None

    def veri_yukle(self):
        """Excel dosyasını yükler ve sütun isimlerindeki boşlukları temizler."""
        try:
            temiz_yol = self.dosya_yolu.replace("\u202a", "").replace("\u202c", "")
            self.orijinal_veri = pd.read_excel(temiz_yol)
            self.orijinal_veri.columns = self.orijinal_veri.columns.str.strip()
            print(f" Veri başarıyla yüklendi. Boyut: {self.orijinal_veri.shape}")
            return self.orijinal_veri
        except FileNotFoundError:
            print(f" HATA: '{self.dosya_yolu}' dosyası bulunamadı.")
            sys.exit()

    def kapsamli_temizlik(self):
        """Veri seti üzerinde tüm temizlik adımlarını uygular."""
        if self.orijinal_veri is None:
            print(" HATA: Önce veri yüklenmelidir. `veri_yukle()` metodunu çalıştırın.")
            return
        
        print("\n Veri temizliği başlıyor...")
        df = self.orijinal_veri.copy()

        kaldirilacak_sutunlar = ['HastaNo', 'KanGrubu']
        df.drop(columns=kaldirilacak_sutunlar, inplace=True, errors='ignore')

        df.dropna(subset=['Bolum'], inplace=True)
        
        if 'UygulamaSuresi' in df.columns:
            df['UygulamaSuresi_Dakika'] = df['UygulamaSuresi'].astype(str).str.extract(r'(\d+)').astype(float)
            medyan_deger = df['UygulamaSuresi_Dakika'].median()
            df['UygulamaSuresi_Dakika'].fillna(medyan_deger, inplace=True)
            df.drop('UygulamaSuresi', axis=1, inplace=True)

        doldurulacak_degerler = {'Alerji': 'Yok', 'KronikHastalik': 'Yok', 'UygulamaYerleri': 'Bilinmiyor'}
        for sutun, deger in doldurulacak_degerler.items():
            if sutun in df.columns:
                df[sutun].fillna(deger, inplace=True)

        if 'Cinsiyet' in df.columns and df['Cinsiyet'].isnull().sum() > 0:
            mod_deger = df['Cinsiyet'].mode()[0]
            df['Cinsiyet'].fillna(mod_deger, inplace=True)

        df.drop_duplicates(inplace=True)
        
        self.hazirlanmis_veri = df
        print(f" Temizlik tamamlandı! Son boyut: {df.shape}")
        return self.hazirlanmis_veri

    def pipeline_olustur_ve_calistir(self, hedef_sutun, test_orani=0.2, random_state=42):
        """Veri işleme pipeline'ını kurar, veriyi eğitir ve dönüştürür."""
        if self.hazirlanmis_veri is None:
            print(" HATA: Önce veri temizlenmelidir. `kapsamli_temizlik()` metodunu çalıştırın.")
            return

        print(f"\n Ön işleme pipeline'ı oluşturuluyor ve çalıştırılıyor...")
        
        X = self.hazirlanmis_veri.drop(columns=[hedef_sutun])
        y = self.hazirlanmis_veri[hedef_sutun]

        sayisal_sutunlar = X.select_dtypes(include=np.number).columns.tolist()
        kategorik_sutunlar = X.select_dtypes(include=['object']).columns.tolist()

        print(f" Sayısal Sütunlar ({len(sayisal_sutunlar)} adet): {sayisal_sutunlar}")
        print(f" Kategorik Sütunlar ({len(kategorik_sutunlar)} adet): {kategorik_sutunlar}")

        sayisal_pipeline = Pipeline(steps=[
            ('doldurucu', SimpleImputer(strategy='median')),
            ('olcekleme', StandardScaler())
        ])

        kategorik_pipeline = Pipeline(steps=[
            ('doldurucu', SimpleImputer(strategy='constant', fill_value='EKSİK_DEGER')),
            ('etiketleyici', GuvenliEtiketleyici())
        ])

        on_isleyici = ColumnTransformer(transformers=[
            ('sayisal', sayisal_pipeline, sayisal_sutunlar),
            ('kategorik', kategorik_pipeline, kategorik_sutunlar)
        ])

        self.tam_pipeline = Pipeline(steps=[('on_isleyici', on_isleyici)])

        X_egitim, X_test, y_egitim, y_test = train_test_split(
            X, y, test_size=test_orani, random_state=random_state, stratify=y
        )
        
        X_egitim_islenmis = self.tam_pipeline.fit_transform(X_egitim)
        X_test_islenmis = self.tam_pipeline.transform(X_test)
        
        print("Pipeline başarıyla çalıştırıldı.")
        
        return X_egitim_islenmis, X_test_islenmis, y_egitim, y_test

    def veri_ozeti_goster(self, df, baslik="Veri Özeti"):
        """DataFrame hakkında basit bir özet bilgi yazdırır."""
        print(f"\n === {baslik.upper()} ===")
        print(f"Boyut: {df.shape}")
        eksik_degerler = df.isnull().sum()
        eksik_deger_sutunlari = eksik_degerler[eksik_degerler > 0]
        if not eksik_deger_sutunlari.empty:
            print("Eksik Değerler:")
            print(eksik_deger_sutunlari)
        else:
            print(" Eksik değer bulunmuyor.")
        print("-" * 30)



In [107]:
# --- PROJE AYARLARI ---
DOSYA_YOLU = r"C:\Users\Enis\Downloads\projectpus.xlsx"
HEDEF_DEGISKEN = 'TedaviSuresi'
TEST_VERI_ORANI = 0.2
RANDOM_STATE = 42
# --------------------


In [108]:
print(" VERİ ÖN İŞLEME PİPELİNE'I BAŞLATILIYOR ")

# 1. Sınıfı başlat ve veriyi yükle
isleyici = VeriOnIsleyici(DOSYA_YOLU)
isleyici.veri_yukle()

# 2. İşlem öncesi veri özetini göster
isleyici.veri_ozeti_goster(isleyici.orijinal_veri, "İşlem Öncesi Veri")

# 3. Veriyi temizle
temiz_veri = isleyici.kapsamli_temizlik()

# 4. Temizlik sonrası veri özetini göster (temiz_veri None değilse)
if temiz_veri is not None:
    isleyici.veri_ozeti_goster(temiz_veri, "Temizlik Sonrası Veri")

    # 5. Pipeline'ı çalıştır ve sonuçları al
    X_egitim_islenmis, X_test_islenmis, y_egitim, y_test = isleyici.pipeline_olustur_ve_calistir(
        hedef_sutun=HEDEF_DEGISKEN,
        test_orani=TEST_VERI_ORANI,
        random_state=RANDOM_STATE
    )

 VERİ ÖN İŞLEME PİPELİNE'I BAŞLATILIYOR 
 Veri başarıyla yüklendi. Boyut: (2235, 13)

 === İŞLEM ÖNCESI VERI ===
Boyut: (2235, 13)
Eksik Değerler:
Cinsiyet           169
KanGrubu           675
KronikHastalik     611
Bolum               11
Alerji             944
Tanilar             75
UygulamaYerleri    221
dtype: int64
------------------------------

 Veri temizliği başlıyor...
 Temizlik tamamlandı! Son boyut: (1277, 11)

 === TEMIZLIK SONRASI VERI ===
Boyut: (1277, 11)
Eksik Değerler:
Tanilar    46
dtype: int64
------------------------------

 Ön işleme pipeline'ı oluşturuluyor ve çalıştırılıyor...
 Sayısal Sütunlar (2 adet): ['Yas', 'UygulamaSuresi_Dakika']
 Kategorik Sütunlar (8 adet): ['Cinsiyet', 'Uyruk', 'KronikHastalik', 'Bolum', 'Alerji', 'Tanilar', 'TedaviAdi', 'UygulamaYerleri']
Pipeline başarıyla çalıştırıldı.


In [115]:
print("\n PİPELİNE TAMAMLANDI! ")
print("=" * 40)
print(" SONUÇ ÖZETİ:")
print(f" İşlenmiş Eğitim Verisi Boyutu: {X_egitim_islenmis.shape}")
print(f" İşlenmiş Test Verisi Boyutu:  {X_test_islenmis.shape}")
print(f" Eğitim Hedef Değişkeni Sayısı: {len(y_egitim)}")
print(f" Test Hedef Değişkeni Sayısı:  {len(y_test)}")
print("\n Artık makine öğrenmesi modellerini eğitebilirsiniz.")
print("   Kullanım: `model.fit(X_egitim_islenmis, y_egitim)`")





 PİPELİNE TAMAMLANDI! 
 SONUÇ ÖZETİ:
 İşlenmiş Eğitim Verisi Boyutu: (1021, 10)
 İşlenmiş Test Verisi Boyutu:  (256, 10)
 Eğitim Hedef Değişkeni Sayısı: 1021
 Test Hedef Değişkeni Sayısı:  256

 Artık makine öğrenmesi modellerini eğitebilirsiniz.
   Kullanım: `model.fit(X_egitim_islenmis, y_egitim)`
